# Membership Inference Attack Examples

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/privML/privacy-evaluator/blob/feat/116-testable-notebooks/notebooks/membership_inference_attack.ipynb"><img src="https://raw.githubusercontent.com/privML/privacy-evaluator/feat/116-testable-notebooks/notebooks/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/privML/privacy-evaluator/blob/feat/116-testable-notebooks/notebooks/membership_inference_attack.ipynb"><img src="https://raw.githubusercontent.com/privML/privacy-evaluator/feat/116-testable-notebooks/notebooks/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview
In this notebook we'll ...

## Setup
First, we should set this notebook's runtime to use a GPU (e.g. if Colab is used go to ***Runtime > Change runtime type > Hardware accelerator***). Now we can install the `privacy-evaluator` package and import all needed modules.

In [ ]:
import numpy as np

import privacy_evaluator.models.torch.dcti.dcti as torch_dcti 
from privacy_evaluator.datasets.cifar10 import CIFAR10
from privacy_evaluator.classifiers.classifier import Classifier
from privacy_evaluator.attacks.membership_inference.black_box import MembershipInferenceBlackBoxAttack
from privacy_evaluator.attacks.membership_inference.black_box_rule_based import MembershipInferenceBlackBoxRuleBasedAttack
from privacy_evaluator.attacks.membership_inference.label_only_decision_boundary import MembershipInferenceLabelOnlyDecisionBoundaryAttack

In [ ]:
!pip3 install git+https://github.com/privML/privacy-evaluator@feat/116-testable-notebooks

## Conduct Membership Inference Attacks

### PyTorch

We start with the PyTorch model. 

#### Load CIFAR10 Dataset

Before we can start to conduct the membership inference attacks, we need to load the dataset. The CIFAR10 dataset needs to be preprocesses in a specific manner to work for the PyTorch model. 

In [ ]:
# Load CIFAR10 dataset as numpy array
x_train, y_train, x_test, y_test = CIFAR10.numpy(model_type='torch')

# Number of classes of CIFAR10 dataset
nb_classes=CIFAR10.N_CLASSES, 

# Input shape of CIFAR10 dataset
input_shape=CIFAR10.INPUT_SHAPE

#### Prepare target model

Now we can set the loss function (in our case it is the `torch.nn.CrossEntropyLoss`) of the model and initialize the model as a generic `Classifier` with the corresponding values.

In [ ]:
# Loss function of the Pytrorch model
torch_loss = torch.nn.CrossEntropyLoss(reduction="none")

# Initalize PyTorch model as a Classifier
torch_target_model = Classifier(
    torch_dcti.load_dcti(), 
    loss=torch_loss, 
    nb_classes=nb_classes, 
    input_shape=input_shape
)

#### Perform Membership Inference Black Box Attack

Prepare, fit and attack

In [ ]:
attack = MembershipInferenceBlackBoxAttack(
    target_model, 
    x_train[:100], 
    y_train[:100], 
    x_test[:100], 
    y_test[:100]
)

attack.fit()
attack.attack(x_train[:100], y_train[:100])

#### Create output

In [ ]:
attack.attack_output(x_train[:100], y_train[:100], np.ones((100,)))

#### Perform Membership Inference Black Box Rule Based Attack

Prepare and attack

In [ ]:
attack = MembershipInferenceBlackBoxRuleBasedAttack(
    target_model, 
    x_train, 
    y_train, 
    x_test, 
    y_test
)

attack.attack(x_train, y_train)

#### Create output

In [ ]:
attack.attack_output(x_train, y_train, np.ones((len(y_train),)))

#### Perform Membership Inference Label Only Decision Boundary Attack

Prepare, fit and attack

In [ ]:
attack = MembershipInferenceLabelOnlyDecisionBoundaryAttack(
    target_model, 
    x_train[:1], 
    y_train[:1], 
    x_test[:1], 
    y_test[:1]
)

attack.fit(max_iter=1, max_eval=1, init_eval=1)
attack.attack(x_train[:1], y_train[:1])

#### Create output

In [ ]:
attack.attack_output(x_train[:1], y_train[:1], np.ones((1,)))